In [1]:
import duckdb
import pandas as pd

In [ ]:
%cd E:\\app_data\\sec_apps_data\\speed_test\\filings_13f_full\\tr_04_filings_parquet_cik
# 1000275-2022-08-15.parquet
df = pd.read_parquet("1000275-2022-08-15.parquet")
 

In [31]:
df.sample(3)

,cik,cusip,quarter,tr_id,tr_type,tr_value,tr_shares,sec_13f_price,value,shares,...,next_shares_diff,next_value_diff,open,max,min,close,pnl_actual_pct,pnl_max_paper_pct,pnl_min_paper_pct,pnl_quarter_pct
122963,1000275,688239201,2006Q1,TR-1000275-688239201-2006Q1,OPEN,5000.0,80.0,62.50,5000.0,80.0,...,0.0,1000.0,62.50,62.50,50.00,50.00,-20.00,0.00,-20.00,0.00
317880,1000275,05644R101,2011Q3,TR-1000275-05644R101-2011Q2,ADD,1000.0,861.0,2.24,4000.0,1786.0,...,NaN,NaN,3.24,3.24,2.24,2.24,-30.86,0.00,-30.86,-30.86
106532,1000275,73936Q835,2011Q3,TR-1000275-73936Q835-2010Q4,REDUCE,-1204000.0,-41404.0,26.35,315000.0,11953.0,...,11223.0,296000.0,27.35,30.67,24.50,27.84,1.79,12.14,-10.42,-3.66


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 345490 entries, 0 to 345489
Data columns (total 43 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   cik                   345490 non-null  Int64         
 1   cusip                 345490 non-null  object        
 2   quarter               345490 non-null  object        
 3   tr_id                 345490 non-null  category      
 4   tr_type               345490 non-null  category      
 5   tr_value              312287 non-null  float64       
 6   tr_shares             312287 non-null  float64       
 7   sec_13f_price         345490 non-null  float64       
 8   value                 345490 non-null  float64       
 9   shares                345490 non-null  float64       
 10  prev_shares_diff      327198 non-null  Float64       
 11  prev_value_diff       327198 non-null  float64       
 12  latest_filed_quarter  345490 non-null  object        
 13 

In [43]:
duckdb.filter(df, "next_shares_diff = 11223.0").df()

,cik,cusip,quarter,tr_id,tr_type,tr_value,tr_shares,sec_13f_price,value,shares,...,next_shares_diff,next_value_diff,open,max,min,close,pnl_actual_pct,pnl_max_paper_pct,pnl_min_paper_pct,pnl_quarter_pct
0,1000275,73936Q835,2011Q3,TR-1000275-73936Q835-2010Q4,REDUCE,-1204000.0,-41404.0,26.35,315000.0,11953.0,...,11223.0,296000.0,27.35,30.67,24.5,27.84,1.79,12.14,-10.42,-3.66


In [65]:
query_str = """
    SELECT cusip, avg(tr_value) as avg_value, quarter
    FROM df
    WHERE year(rdate) NOT IN (2020, 2021)
    GROUP BY cusip, quarter
    HAVING abs(avg(tr_value)) < 10000
    LIMIT 10
"""
    # WHERE year(rdate) IN (2020, 2021)
    # GROUP BY cusip
    # HAVING avg(tr_value) > 1000000
# duckdb.query(query_str)
duckdb.query(query_str).df()
out = duckdb.query(query_str).to_df()

In [66]:
out.sample()

,cusip,avg_value,quarter
8,09253X102,2000.0,2015Q2
